## Example 1. Rigid Body Rotation: Euler's Equations

This notebook solves the Euler equations by different RK methods and their relaxation versions:
\begin{align}
    \dot{u}_{1}=(\alpha - \beta)u_2u_3 \\
    \dot{u}_{2}=(1-\alpha)u_3u_1 \\
    \dot{u}_{3}=(\beta-1)u_1u_2\;,
\end{align}
with $\left(u_1(0),u_2(0),u_3(0)\right)^T = (0,1,1)^T$, $\alpha = 1 + \frac{1}{\sqrt{1.51}}$, and $\beta = 1 - \frac{0.51}{\sqrt{1.51}}$. The exact solution is
    \begin{align}
        \left(u_1(t),u_2(t),u_3(t)\right)^T = \left(\sqrt{1.51} \  \textrm{sn}(t,0.51),\textrm{cn}(t,0.51),\textrm{dn}(t,0.51)\right)^T \;,
    \end{align}
    where $\textrm{sn},\ \textrm{cn},\ \text{and}, \ \textrm{dn}$ are the elliptic Jacobi functions. This problem has two quadratic conserved quantities:
\begin{align}
    & G_1(u_{1},u_{2},u_{3}) = u_{1}^{2}+ u_{2}^{2}+u_{3}^{2}\\
    & G_2(u_{1},u_{2},u_{3}) = u_{1}^{2}+ \beta u_{2}^{2}+\alpha u_{3}^{2} \;.
\end{align}

In [ ]:
#Required libraries 
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from nodepy import rk
from IPython.display import clear_output
from RKSchemes import ssp22, heun33, ssp33, rk44, fehlberg45, dp75

In [ ]:
# Required functions for Euler's equation
def EulerEqs_f(u):
    u1 = u[0]; u2 = u[1]; u3 = u[2];
    alpha = 1 + 1/np.sqrt(1.51); beta = 1 - 0.51/np.sqrt(1.51)
    du1 = (alpha - beta)*u2*u3
    du2 = (1 - alpha)*u3*u1
    du3 = (beta - 1)*u1*u2
    return np.array([du1, du2, du3])

def G_1(u):
    u1 = u[0]; u2 = u[1]; u3 = u[2];
    return u1*u1 + u2*u2 + u3*u3

def G_2(u):
    u1 = u[0]; u2 = u[1]; u3 = u[2];
    alpha = 1 + 1/np.sqrt(1.51); beta = 1 - 0.51/np.sqrt(1.51)
    return u1*u1 + beta*u2*u2 + alpha*u3*u3

def rgam(gammas,u,inc1,inc2,E1_old,E2_old):
    gamma1, gamma2 = gammas
    uprop = u + gamma1*inc1 + gamma2*inc2  
    E1 = G_1(uprop)
    E2 = G_2(uprop)
    return np.array([E1-E1_old,E2-E2_old])

from scipy import special
def EulerEqs_ExactSol(t):
    sn, cn, dn, ph = special.ellipj(t, 0.51) 
    u1 = np.sqrt(1.51)*sn
    u2 = cn
    u3 = dn
    return np.array([u1, u2, u3])

### Baseline RK methods

In [ ]:
# Compute solution with baseline RK methods
def compute_sol_without_relaxation(Mthdname,rkm, dt, f, T, u0,t0): 
    tt = np.zeros(1) 
    t = t0; tt[0] = t
    uu = np.zeros((1,np.size(u0))) 
    uu[0,:] = u0.copy()
    
    s = len(rkm)
    y = np.zeros((s,len(u0))) 
    F = np.zeros((s,len(u0))) 
    steps = 0
    
    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        if t + dt > T:
            dt = T - t
        for i in range(s):
            y[i,:] = uu[-1].copy()
            for j in range(i):
                y[i,:] += dt*rkm.A[i,j]*F[j,:]
            F[i,:] = f(y[i,:])
        inc = dt*sum([rkm.b[i]*F[i] for i in range(s)])    
        unew = uu[-1]+inc; t+= dt
        tt = np.append(tt, t)
        steps +=1
        uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0)  
        print("Method = Baseline %s: Step number = %d (time = %1.2f)"%(Mthdname,steps,tt[-1]))
    return tt, uu

### Relaxation RK methods

In [ ]:
# Computing solution with multiple relaxation methods
def compute_sol_multi_relaxation(Mthdname,rkm, dt, f, T, u0, t0):
    tt = np.zeros(1) 
    t = t0; tt[0] = t
    uu = np.zeros((1,np.size(u0))) 
    uu[0,:] = u0.copy()
    
    s = len(rkm)
    y = np.zeros((s,len(u0))) 
    F = np.zeros((s,len(u0))) 
    
    G1 = np.array([]); G2 = np.array([])
    no_inv = 2; gammas0 = np.zeros(no_inv)
    
    errs = 0; steps = 0
    
    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        if t + dt > T:
            dt = T - t
        for i in range(s):
            y[i,:] = uu[-1].copy()
            for j in range(i):
                y[i,:] += dt*rkm.A[i,j]*F[j,:]
            F[i,:] = f(y[i,:])
            
        inc1 = dt*sum([rkm.b[i]*F[i] for i in range(s)])
        inc2 = dt*sum([rkm.bhat[i]*F[i] for i in range(s)])
        
        wr_unew = uu[-1] + inc1; E1_old = G_1(uu[-1]); E2_old = G_2(uu[-1])
        
        gammas, info, ier, mesg = fsolve(rgam,gammas0,args=(wr_unew,inc1,inc2,E1_old,E2_old),full_output=True)
        gamma1, gamma2 = gammas
        
        unew =  wr_unew + gamma1*inc1 + gamma2*inc2; t+=(1+gamma1+gamma2)*dt
        tt = np.append(tt, t)
        steps += 1
        uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0) 
        G1 = np.append(G1, gamma1); G2 = np.append(G2, gamma2)
        print("Method = Relaxation %s: At step number = %d (time = %1.2f), integer flag = %d and γ1+γ2 = %f \n"%(Mthdname,steps,tt[-1],ier,gamma1+gamma2))

    return tt, uu, G1, G2

### Compute solution by all methods

In [ ]:
%time
eqn = 'EE'
methods = [heun33,fehlberg45,dp75]
method_labels = ["Heun(3,3)", "Fehlberg(6,4)", "DP5(7,5)"]
method_names = ["Heuns3p3", "Fehlbergs6p4", "DP5s7p5"]

# Inputs to solve the system of ODEs
DT = [0.04, .1, .1];  f = EulerEqs_f; T = 1000; 

# Initial condition
t0 = 0; u0 = np.array([0.0, 1.0, 1.0])
b_tt = []; b_uu = []; r_tt = []; r_uu = []  # empty list to store data by all the methods
    
for idx in range(len(methods)):
    print(idx)
    rkm = methods[idx]; dt = DT[idx]
    tt, uu, G1, G2 = compute_sol_multi_relaxation(method_labels[idx], rkm, dt, f, T, u0, t0)
    r_tt.append(tt); r_uu.append(uu)
    
for idx in range(len(methods)):
    rkm = methods[idx]; dt = DT[idx]
    tt, uu = compute_sol_without_relaxation(method_labels[idx], rkm, dt , f, T, u0, t0)
    b_tt.append(tt); b_uu.append(uu)
    


In [ ]:
import os
path = '%s'%('Figures')

import os
if not os.path.exists(path):
   os.makedirs(path)

### Compute and plot changes in invariants by different methods

In [ ]:
b_G_1 = []; b_G_2 = []; r_G_1 = []; r_G_2 = []
for i in range(len(methods)):
    b_g_1 = [G_1(u) for u in b_uu[i]] - G_1(b_uu[i][0])
    b_g_2 = [G_2(u) for u in b_uu[i]] - G_2(b_uu[i][0])
    r_g_1 = [G_1(u) for u in r_uu[i]] - G_1(r_uu[i][0])
    r_g_2 = [G_2(u) for u in r_uu[i]] - G_2(r_uu[i][0])
    b_G_1.append(b_g_1); b_G_2.append(b_g_2)
    r_G_1.append(r_g_1); r_G_2.append(r_g_2)

In [ ]:
methods = [heun33,fehlberg45,dp75]
method_labels = ["Heun(3,3)", "Fehlberg(6,4)", "DP5(7,5)"]
method_names = ["Heuns3p3", "Fehlbergs6p4", "DP5s7p5"]

font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 14}
plt.rc('font', **font) 
plt.figure(figsize=(15, 4))
for i in range(len(methods)):
    plt.subplot(1,len(methods),i+1)
    plt.plot(b_tt[i],b_G_1[i],':r',label="Baseline: $G_1(u_1(t),u_2(t),u_3(t))-G_1(u_1(0),u_2(0),u_3(0))$")
    plt.plot(r_tt[i],r_G_1[i],'-r',label="Relaxation: $G_1(u_1(t),u_2(t),u_3(t))-G_1(u_1(0),u_2(0),u_3(0))$")
    plt.plot(b_tt[i],b_G_2[i],':b',label="Baseline: $G_2(u_1(t),u_2(t),u_3(t))-G_2(u_1(0),u_2(0),u_3(0))$")
    plt.plot(r_tt[i],r_G_2[i],'-b',label="Relaxation: $G_2(u_1(t),u_2(t),u_3(t))-G_2(u_1(0),u_2(0),u_3(0))$")
    plt.title("%s with $\Delta t$ = %.2f"%(method_labels[i],DT[i]))
    plt.xlabel("$t$");
    #plt.ylabel("$\eta(u(t)) - \eta(u^0)$")
    plt.yscale("symlog", linthresh=1.e-14)
    plt.yticks([-1.e-6, -1.e-10, -1.e-14, 1.e-14, 1.e-10, 1.e-6])
    

plt.tight_layout()
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper center', ncol=2, bbox_to_anchor=(0.5, 1.2))
#plt.show()
plt.savefig("./Figures/Rigid_Body_2_Inv_Error_Time.pdf", bbox_inches="tight")

### Compute and plot solution errors by different methods

In [ ]:
b_ERR = []; r_ERR = []
for i in range(len(methods)):
    # maximum norm
    b_err = [np.max(np.abs(b_uu[i][j] - EulerEqs_ExactSol(b_tt[i][j]))) for j in np.arange(len(b_tt[i]))]
    r_err = [np.max(np.abs(r_uu[i][j] - EulerEqs_ExactSol(r_tt[i][j]))) for j in np.arange(len(r_tt[i]))]
    b_ERR.append(b_err); r_ERR.append(r_err)

In [ ]:
methods = [heun33,fehlberg45,dp75]
method_labels = ["Heun(3,3)", "Fehlberg(6,4)", "DP5(7,5)"]
method_names = ["Heuns3p3", "Fehlbergs6p4", "DP5s7p5"]

lgd_box_pos = [[0.4,0.2,0.5, 0.5],[0.4,0.1,0.5, 0.5],[0.4,0.5,0.5, 0.5]]
sl1_cons_mult = [5e-7,5e-8,3e-9]; sl1_p = [1,1,1]
sl2_cons_mult = [8*1e-8,9*1e-10,3*1e-11]; sl2_p = [2,2,2]
y_scale_line = [1e-10,1e-8,1e-7]

font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 14}
plt.rc('font', **font)
shift = 1
plt.figure(figsize=(15, 4))

for i in range(len(methods)):
    plt.subplot(1,len(methods),i+1)
    plt.plot(b_tt[i]+shift,b_ERR[i],':',color='orangered',label="Baseline")
    plt.plot(r_tt[i]+shift,r_ERR[i],'-g',label="Relaxation")
    sl_t = np.linspace(10,T,1000)

    plt.plot(sl_t,sl1_cons_mult[i]*sl_t**sl1_p[i],'--',color='0.5',label="$\mathcal{O}(t^{%d})$"%(sl1_p[i]))
    plt.plot(sl_t,sl2_cons_mult[i]*sl_t**sl2_p[i],'-',color='0.5',label="$\mathcal{O}(t^{%d})$"%(sl2_p[i]))
     
    plt.xscale("log"); plt.yscale("log")
    plt.xlabel('t'); plt.ylabel('Error in q')
    plt.title("%s with $\Delta t$ = %.2f"%(method_labels[i],DT[i]))
    plt.tight_layout()
    
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper center', ncol=6, bbox_to_anchor=(0.5, 1.1))
#plt.show()
plt.savefig("./Figures/Rigid_Body_Sol_Err_Time.pdf", bbox_inches="tight")

### Compute errors and plot error convergence

In [ ]:
methods = [ssp22,heun33,rk44,dp75]
method_names = ["ssps2p2","heuns3p3", "rks4p4", "DP5s7p5"]
dts = 1/2**np.linspace(3,8,10)
dts = dts[0:]
f = EulerEqs_f; tf = 5
# Initial condition
t0 = 0; u0 = np.array([0.0, 1.0, 1.0])
b_errs = np.zeros((len(methods),len(dts))); r_errs = np.zeros((len(methods),len(dts)))

for idx in range(len(methods)):
    rkm = methods[idx]
    for dt_idx in range(len(dts)):
        dt = dts[dt_idx]
        b_T, b_U = compute_sol_without_relaxation(method_names[idx], rkm, dt, f, tf, u0, t0)
        r_T, r_U, G1, G2 = compute_sol_multi_relaxation(method_names[idx], rkm, dt, f, tf, u0, t0)
        
        b_error = np.max(np.abs(b_U[-1]-EulerEqs_ExactSol(b_T[-1]))); 
        r_error = np.max(np.abs(r_U[-1]-EulerEqs_ExactSol(r_T[-1]))); 
        b_errs[idx][dt_idx]=b_error; r_errs[idx][dt_idx]=r_error; 

In [ ]:
# Plotting routine
from itertools import cycle
lines = [":","--","-.","-"]
colors = ["red", "blue", "green" ,"darkviolet"]
method_labels = ["SSPRK(2,2)","Heun(3,3)", "RK(4,4)", "DP(7,5)"]

# Font size 
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 12}
plt.rc('font', **font) 

fig = plt.figure(1)
for idx in range(len(methods)):
    plt.plot(dts,b_errs[idx],color = colors[idx],linestyle=lines[idx],label = method_labels[idx])
    
plt.plot(dts, 1.5e-0*dts**2, "-.", color="gray")
plt.annotate(r"$\mathcal{O}(t^2)$", (2.0e-2, 1.0e-3), color="gray")
plt.plot(dts, .5e-0*dts**3, "-P", color="gray")
plt.annotate(r"$\mathcal{O}(t^3)$", (2.0e-2, 1.2e-5), color="gray")
plt.plot(dts, 1e-1*dts**4, "--s", color="gray")
plt.annotate(r"$\mathcal{O}(t^4)$", (2.0e-2, 8.0e-8), color="gray")
plt.plot(dts[:8], .5e-2*dts[:8]**5, ":o", color="gray")
plt.annotate(r"$\mathcal{O}(t^5)$", (2.0e-2, 1.0e-10), color="gray")
plt.xscale("log"); plt.yscale("log"); plt.xlabel("$\Delta t$"); plt.ylabel('Error')
fig.tight_layout()
plt.savefig('./Figures/%s_Baseline_Mthds_ErrorConv_tf%d.pdf'%(eqn,tf),format='pdf', bbox_inches="tight")

fig = plt.figure(2)
for idx in range(len(methods)):
    plt.plot(dts,r_errs[idx],color = colors[idx],linestyle=lines[idx],label = method_labels[idx])

plt.plot(dts, 1.5e-0*dts**2, "-.", color="gray")
plt.annotate(r"$\mathcal{O}(t^2)$", (2.0e-2, 2.0e-3), color="gray")
plt.plot(dts, 2.0e-1*dts**3, "-P", color="gray")
plt.annotate(r"$\mathcal{O}(t^3)$", (2.0e-2, 1.0e-5), color="gray")
plt.plot(dts, 1.5e-1*dts**4, "--s", color="gray")
plt.annotate(r"$\mathcal{O}(t^4)$", (2.0e-2, 8.0e-8), color="gray")
plt.plot(dts[:8], 1e-2*dts[:8]**5, ":o", color="gray")
plt.annotate(r"$\mathcal{O}(t^5)$", (2.0e-2, 1.0e-10), color="gray")
plt.xscale("log"); plt.yscale("log"); plt.xlabel("$\Delta t$"); plt.ylabel('Error')
fig.tight_layout()
plt.savefig('./Figures/%s_Relaxation_Mthds_ErrorConv_tf%d.pdf'%(eqn,tf),format='pdf', bbox_inches="tight")